In [1]:
import open3d as o3d
import numpy as np
import open3d.visualization as vis

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Load the original mesh (150 Mb), convert it into a point cloud, then save as numpy array

In [82]:
mesh_path = "obj_files/box_with_texture_aligned.obj"

mesh = o3d.io.read_triangle_mesh(mesh_path)
mesh.compute_vertex_normals()

print(mesh)

TriangleMesh with 6325486 points and 2110464 triangles.


### This mesh will be downsampled to exactly 4096 with different methods which are then compared 

##### 1. Downsample mesh using uniform sampling --> uniformly samples points from the surface triangles

In [29]:
pcd = mesh.sample_points_uniformly(number_of_points=4096)
o3d.visualization.draw_geometries([pcd])

The result looks fine although many points just represent bare surface points and important corners or edges are not really emphasized. Also the method may yield clusters on the surface. To avoid these Poisson disk sampling can be used.

### 2. Downsample mesh using Poisson Disk Sampling
distributes the point on the points evenly on the surface

In [28]:
pcd = mesh.sample_points_poisson_disk(number_of_points=4096, init_factor=5)
o3d.visualization.draw_geometries([pcd])

The result definitely has more evenly distributes than before but corners and edges are still not emphasiszed.

### As sampling may not yield the besult results regarding the requirement, instead mesh simplification methods will be tried. The result will still be a mesh but can then be simply converted into a point cloud
### 3. The first one being vertex clustering: vertices which fall into a voxel of a given size are treated as one vertice which is computed e.g. by averaging


In [20]:
print(
    f'Input mesh has {len(mesh.vertices)} vertices and {len(mesh.triangles)} triangles'
)

Input mesh has 6325486 vertices and 2110464 triangles


In [42]:
voxel_size = max(mesh.get_max_bound() - mesh.get_min_bound()) / 32
print(f'voxel_size = {voxel_size:e}')
mesh = mesh.simplify_vertex_clustering(
    voxel_size=voxel_size,
    contraction=o3d.geometry.SimplificationContraction.Average)
print(
    f'Simplified mesh has {len(mesh.vertices)} vertices and {len(mesh.triangles)} triangles'
)

# Simplify the mesh with a different contraction method --> quadratic error minimized the distance to the adjacent triange planes
mesh_q = mesh.simplify_vertex_clustering(
    voxel_size=voxel_size,
    contraction=o3d.geometry.SimplificationContraction.Quadric)

# convert mesh to point cloud and visualize without sampling
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = mesh.vertices

# convert mesh to point cloud and visualize without sampling
point_cloud_q = o3d.geometry.PointCloud()
point_cloud_q.points = mesh.vertices


# put the original mesh and the simplified mesh in the same window but seperate by 10 cm
point_cloud_q.translate(np.array([0, 0, -0.1]))

o3d.visualization.draw_geometries([point_cloud, point_cloud_q])

voxel_size = 4.870375e-03
[Open3D WARNING] [SimplifyVertexClustering] This mesh contains triangle uvs that are not handled in this function
Simplified mesh has 3226 vertices and 6642 triangles


The result has perfectly distributed points and also the corners are well visible 

In [34]:
# convert to point cloud and numpy array
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = mesh.vertices

point_cloud_np = np.asarray(point_cloud.points)
print("PointCloud converted to numpy array:")
print(point_cloud_np.shape)

# print first 10 points
print(point_cloud_np[:10])

# save numpy array to file
np.save("point_clouds/point_cloud.npy", point_cloud_np)

PointCloud converted to numpy array:
(6325486, 3)
[[-0.007254 -0.045732 -0.001409]
 [-0.007352 -0.045725 -0.001605]
 [-0.007089 -0.045722 -0.001752]
 [-0.007089 -0.045722 -0.001752]
 [-0.006969 -0.045728 -0.001512]
 [-0.007254 -0.045732 -0.001409]
 [-0.007196 -0.045735 -0.001294]
 [-0.007254 -0.045732 -0.001409]
 [-0.006969 -0.045728 -0.001512]
 [-0.006969 -0.045728 -0.001512]]


### Load the numpy array, convert to point cloud and visualize it

In [17]:
# load the numpy array
point_cloud_np = np.load("point_clouds/point_cloud.npy")

# convert to open3d point cloud
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(point_cloud_np)

# visualize the point cloud
vis.draw_geometries([point_cloud])

In [83]:
# compute normals of original mesh
mesh.compute_vertex_normals()
mesh.compute_triangle_normals()

# iterate over all vertices and print the neighbors of the first vertex
vertex_id = 0
print(f"Neighbors of vertex {vertex_id}:")
print(mesh.vertex_normals[vertex_id])
print(mesh.vertex_normals[vertex_id-1])
print(mesh.vertex_normals[vertex_id+1])

angles = np.zeros(len(mesh.vertex_normals))

for i in range(len(mesh.vertex_normals)):
    if i+1 < len(mesh.vertex_normals):
        angle_1 = np.abs(np.arccos(np.dot(mesh.vertex_normals[i-1], mesh.vertex_normals[i])))
        angle_2 = np.abs(np.arccos(np.dot(mesh.vertex_normals[i+1], mesh.vertex_normals[i])))
        # save the bigger angle
        angles[i] = max(angle_1, angle_2)
        
print(angles.shape)

Neighbors of vertex 0:
[-0.00840003 -0.99940341 -0.03350011]
[ 0.82753637 -0.54152378 -0.14810652]
[-0.00840003 -0.99940341 -0.03350011]


/tmp/ipykernel_23745/3951133516.py:17: RuntimeWarning: invalid value encountered in arccos
  angle_2 = np.arccos(np.dot(mesh.vertex_normals[i+1], mesh.vertex_normals[i]))
/tmp/ipykernel_23745/3951133516.py:16: RuntimeWarning: invalid value encountered in arccos
  angle_1 = np.arccos(np.dot(mesh.vertex_normals[i-1], mesh.vertex_normals[i]))


KeyboardInterrupt: 

In [92]:
# Assuming 'mesh' is already defined and is a valid mesh object
mesh.compute_vertex_normals()

# No need to explicitly compute and print neighbors for efficiency in this context

# Precompute the cosines for all normal pairs to avoid redundant trigonometric calculations
cosines = np.einsum('ij,ij->i', mesh.vertex_normals[:-1], mesh.vertex_normals[1:])
# Compute angles using arccos, note that we clip values to be within the valid domain of arccos
angles = np.arccos(np.clip(cosines, -1.0, 1.0))

# Initialize an array to store the maximum angle for each vertex
max_angles = np.zeros(len(mesh.vertex_normals))

# For the interior vertices, compare adjacent angles and take the maximum
max_angles[1:-1] = np.maximum(angles[:-1], angles[1:])

# For the boundary vertices, we only consider one neighbor
max_angles[0] = angles[0]
max_angles[-1] = angles[-1]

print(max_angles.shape)

(6325486,)


In [106]:
# keep all vertices with an angle bigger than 30 degrees
threshold = np.deg2rad(0.1)
vertices_to_keep = np.where(max_angles > threshold)[0]

print(f"Keeping {len(vertices_to_keep)} vertices out of {len(mesh.vertices)}")

Keeping 4071931 vertices out of 6325486


In [107]:

# # create a new mesh with only the vertices that have an angle bigger than 60 degrees
# mesh_filtered = mesh.select_by_index(vertices_to_keep)

# Placeholder for the new faces list
new_faces = []

# Mapping from old vertex indices to new indices
old_to_new_indices = {old_idx: new_idx for new_idx, old_idx in enumerate(vertices_to_keep)}

for face in mesh.triangles:
    # Check if all vertices of the face should be kept
    if all(vertex in vertices_to_keep for vertex in face):
        # If so, add the face with updated vertex indices to the new faces list
        new_faces.append([old_to_new_indices[vertex] for vertex in face])

# Create a new mesh with the filtered vertices and faces
mesh_filtered = o3d.geometry.TriangleMesh()
mesh_filtered.vertices = o3d.utility.Vector3dVector(np.asarray(mesh.vertices)[vertices_to_keep])
mesh_filtered.triangles = o3d.utility.Vector3iVector(new_faces)
mesh_filtered.compute_vertex_normals()  # Recalculate normals for the new mesh


# visualize the filtered mesh
o3d.visualization.draw_geometries([mesh_filtered])

KeyboardInterrupt: 

In [105]:
# convert the filtered mesh to a point cloud
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = mesh_filtered.vertices

# visualize the point cloud
o3d.visualization.draw_geometries([point_cloud])

In [17]:
# load numpy array from disk
point_cloud_np = np.load("point_clouds/my_function_result.npy")
print(point_cloud_np.shape)

# convert to point cloud and plot
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(point_cloud_np)
o3d.visualization.draw_geometries([point_cloud])


(4096, 3)


#### Normalize the point cloud

In [18]:
print(f"Largest x value: {np.max(point_cloud_np[:, 0])}")
print(f"Smallest x value: {np.min(point_cloud_np[:, 0])}")

print(f"Largest y value: {np.max(point_cloud_np[:, 1])}")
print(f"Smallest y value: {np.min(point_cloud_np[:, 1])}")

print(f"Largest z value: {np.max(point_cloud_np[:, 2])}")
print(f"Smallest z value: {np.min(point_cloud_np[:, 2])}")

Largest x value: 0.07798146820495422
Smallest x value: -0.07755728280417626
Largest y value: 0.046481068120535284
Smallest y value: -0.045768397148182906
Largest z value: 0.04597732768171476
Smallest z value: -0.04611275701219557


In [19]:
# check if the points in x are centered, center it
centroid_x = np.mean(point_cloud_np[:, 0])
centroid_y = np.mean(point_cloud_np[:, 1])
centroid_z = np.mean(point_cloud_np[:, 2])
centroid = np.array([centroid_x, centroid_y, centroid_z])


if not np.allclose(centroid, np.zeros(3), atol=1e-6):
    print(f"Centering Point Cloud around {centroid}!")
    new_point_cloud_np = point_cloud_np - centroid
    
# convert to point cloud and plot
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(new_point_cloud_np)
o3d.visualization.draw_geometries([point_cloud])
   


# check if the point cloud is normalized
norms = np.linalg.norm(new_point_cloud_np, axis=1)  # Calculate the Euclidean norm for each point
max_distance = np.max(norms)
print(f"Maximum distance from origin: {max_distance}")
if not max_distance <= 1:
    print("Normalizing Point Cloud!")
    new_point_cloud_np = new_point_cloud_np / max_distance
    
# convert to point cloud and plot
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(new_point_cloud_np)
o3d.visualization.draw_geometries([point_cloud])

Centering Point Cloud around [ 0.00034515 -0.00016372  0.00057998]!
Maximum distance from origin: 0.09872504121963158
